In [1]:
# # Cell 1: Install All Required Libraries
# %%
!pip install -U -q numpy pandas tensorflow scikit-learn nltk praw mealpy joblib
print("All libraries installed.")

All libraries installed.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install -U numpy pandas


  Using cached numpy-2.3.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-2.3.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mealpy 3.0.3 requires numpy<=1.26.0,>=1.17.1, but you have numpy 2.3.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompatible.
tf-keras 2.19.0 requires tensorflo

In [2]:
# # Cell 2: Import All Libraries
# %%
import os
import re
import json
import string
import warnings
import numpy as np
import pandas as pd
import nltk
import joblib
import time
from tqdm import tqdm

# NLTK
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer, tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Scikit-learn
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split



# Import the correct AOA class
from mealpy.math_based.AOA import OriginalAOA
# --- THIS IS THE MISSING IMPORT ---
from mealpy.utils.space import IntegerVar
# ---

# Suppress TensorFlow/Keras warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings
warnings.filterwarnings('ignore')

print("Mealpy (AOA) and Joblib installed.")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

print("All core libraries imported.")

Mealpy (AOA) and Joblib installed.
All core libraries imported.


# Phase 1

In [3]:

# # Cell 3: ACTION REQUIRED - Upload Your Dataset Files
# %%
from google.colab import files

print("Please upload your 'train.tsv', 'dev.tsv', and 'test.tsv' files.")
uploaded = files.upload()

# --- Verification ---
if 'train.tsv' in uploaded and 'dev.tsv' in uploaded and 'test.tsv' in uploaded:
    print("\nAll three files uploaded successfully!")
    print(f"train.tsv size: {len(uploaded['train.tsv'])} bytes")
    print(f"dev.tsv size: {len(uploaded['dev.tsv'])} bytes")
    print(f"test.tsv size: {len(uploaded['test.tsv'])} bytes")
    print("\nProceed to the next cell.")
else:
    print("\n--- ERROR ---")
    print("One or more files are missing. Please re-run this cell and upload all three .tsv files.")

ModuleNotFoundError: No module named 'google.colab'

In [3]:

# # Cell 4: Global Parameters & NLTK Download
# %%
# Download NLTK assets
print("Downloading NLTK assets...")
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt_tab', quiet = True)
print("NLTK assets downloaded.")

# Define Global Model Parameters
EMOTION_LABELS = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
    'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love',
    'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse',
    'sadness', 'surprise', 'neutral'
]
NUM_LABELS = len(EMOTION_LABELS)
VOCAB_SIZE = 20000  # Max number of words
MAX_LEN = 100       # Max length of a comment

# Create and fit the MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=range(NUM_LABELS))
mlb.fit(None)
print(f"Model configured for {NUM_LABELS} emotion labels.")

NLTK assets downloaded.
Model configured for 28 emotion labels.


In [4]:
# ---
# ## Cell 4: Load & Preprocess GoEmotions Data
# ---
# ### 4.1. Define Text Cleaning Functions
# %%
# Initialize NLTK tools (we do this once to be efficient)
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    """Cleans text data for processing."""
    text = str(text).lower()  # Lowercase
    text = re.sub(r'http\S+|www\S+', ' ', text)  # Remove URLs
    text = re.sub(r'@[^\s]+', ' ', text)  # Remove mentions
    text = re.sub(r'\[name\]|\[religion\]', ' ', text) # Remove GoEmotions tags
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)  # Remove punctuation and numbers

    # Tokenize
    tokens = word_tokenize(text)

    cleaned_tokens = []
    for word in tokens:
        # Remove stopwords and short words
        if word not in stop_words and len(word) > 2:
            cleaned_tokens.append(lemmatizer.lemmatize(word))

    return ' '.join(cleaned_tokens)

def process_labels(label_str):
    """Converts a label string '10,14,20' into a 28-element multi-hot vector."""

    if not isinstance(label_str, str) or label_str.strip() == '':
        label_indices = []  # no labels
    else:
        # The .tsv file uses commas, not semicolons
        label_indices = [int(i) for i in label_str.split(',') if i.isdigit()]

    # --- FIX: Removed one pair of square brackets ---
    # Use the pre-fitted MLB to transform this list into a multi-hot vector
    label_vector = mlb.transform([label_indices])[0]
    # --- END FIX ---

    return label_vector

# ### 4.2. Load, Clean, and Process Data
# %%
print("Loading and preprocessing GoEmotions data...")

# Load data
try:
    # --- Read .tsv files using sep='\t' and assign column names ---
    col_names = ['text', 'labels', 'id']
    df_train_raw = pd.read_csv('train.tsv', sep='\t', header=None, names=col_names)
    df_val_raw = pd.read_csv('dev.tsv', sep='\t', header=None, names=col_names)
    df_test_raw = pd.read_csv('test.tsv', sep='\t', header=None, names=col_names)

except FileNotFoundError:
    print("\n--- !!! CRITICAL ERROR !!! ---")
    print("One or more .tsv files were not found. Did you download and unzip them in Cell 3?")
    print("Please re-run Cell 3 or check your internet connection.")
    raise

# --- VALIDATION STEP ---
# The training file should have > 40,000 rows.
if df_train_raw.shape[0] < 1000:
    print("\n--- !!! CRITICAL ERROR !!! ---")
    print(f"Data loading failed. Only found {df_train_raw.shape[0]} rows.")
    print("The .tsv files likely contain error messages instead of data.")
    print("Please re-run Cell 3.")
    raise Exception("Data Load Error: TSV files are not valid.")
else:
    print(f"Successfully loaded {df_train_raw.shape[0]} training rows.")
# --- END VALIDATION STEP ---


# For our "Base" model, we'll combine the train and validation sets
# This gives us more data to build a strong "expert"
df_base_train = pd.concat([df_train_raw, df_val_raw], ignore_index=True)
df_base_test = df_test_raw

# Apply text cleaning
print("Cleaning text data...")
df_base_train['cleaned_text'] = df_base_train['text'].apply(clean_text)
df_base_test['cleaned_text'] = df_base_test['text'].apply(clean_text)

# Apply label processing (using the 'labels' column)
print("Processing labels...")
y_train_base = np.array(df_base_train['labels'].apply(process_labels).tolist())
y_test_base = np.array(df_base_test['labels'].apply(process_labels).tolist())

# Assign cleaned text to X variables
X_train_base = df_base_train['cleaned_text']
X_test_base = df_base_test['cleaned_text']

print("\n--- Data Preprocessing Complete ---")
print(f"Base Training data shape: {X_train_base.shape}")
print(f"Base Training labels shape: {y_train_base.shape}")
print(f"Base Test data shape: {X_test_base.shape}")
print(f"Base Test labels shape: {y_test_base.shape}")

print("\nSample processed text:", X_train_base.iloc[0])
print("Sample processed label vector:", y_train_base[0])

Loading and preprocessing GoEmotions data...
Successfully loaded 43410 training rows.
Cleaning text data...
Processing labels...

--- Data Preprocessing Complete ---
Base Training data shape: (48836,)
Base Training labels shape: (48836, 28)
Base Test data shape: (5427,)
Base Test labels shape: (5427, 28)

Sample processed text: favourite food anything cook
Sample processed label vector: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [5]:
# ---
# ## Cell 5: Text Vectorization (Tokenizer)
# ---

# Now we convert our cleaned text sentences into sequences of numbers that the model can understand.
# %%
print("Fitting Tokenizer on base training data...")

# Initialize and fit the tokenizer
# This builds a "word-to-index" dictionary (e.g., {'the': 1, 'good': 2, ...})
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_base)

# Convert text to sequences of numbers
print("Vectorizing and padding sequences...")
X_train_pad = pad_sequences(tokenizer.texts_to_sequences(X_train_base), maxlen=MAX_LEN, padding='post', truncating='post')
X_test_pad = pad_sequences(tokenizer.texts_to_sequences(X_test_base), maxlen=MAX_LEN, padding='post', truncating='post')

# Save the tokenizer to a file. We will need this exact tokenizer for all future steps.
tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

print("\n--- Tokenization Complete ---")
print("Tokenizer saved to 'tokenizer.json'")
print(f"Padded training sequences shape: {X_train_pad.shape}")
print(f"Padded test sequences shape: {X_test_pad.shape}")
print("\nSample vectorized sequence:")
print(X_train_pad[0])

Fitting Tokenizer on base training data...
Vectorizing and padding sequences...

--- Tokenization Complete ---
Tokenizer saved to 'tokenizer.json'
Padded training sequences shape: (48836, 100)
Padded test sequences shape: (5427, 100)

Sample vectorized sequence:
[ 773  329   99 2809    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [6]:

# ---
# ## Cell 6: Build & Train Base CNN Model
# ---

# ### 6.1. Define the CNN Architecture
#
# This is our "Emotion Expert" model. It's a 1D CNN optimized for multi-label text classification.
# %%
def build_base_cnn_model(vocab_size=VOCAB_SIZE, max_len=MAX_LEN, num_labels=NUM_LABELS):
    """Builds the Keras CNN model for multi-label classification."""

    inputs = Input(shape=(max_len,))

    # Embedding layer: Turns word indices (e.g., 5) into dense vectors (e.g., [0.1, -0.4, ...])
    embedding = Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len)(inputs)

    # Convolutional layers: Scan for patterns of different sizes (3, 4, and 5 words)
    conv1 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(embedding)
    pool1 = GlobalMaxPooling1D()(conv1)

    conv2 = Conv1D(filters=128, kernel_size=4, activation='relu', padding='same')(embedding)
    pool2 = GlobalMaxPooling1D()(conv2)

    conv3 = Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(embedding)
    pool3 = GlobalMaxPooling1D()(conv3)

    # Concatenate all the features found
    merged = tf.keras.layers.concatenate([pool1, pool2, pool3], axis=1)

    # Fully connected layers for classification
    dense1 = Dense(256, activation='relu')(merged)
    dense1 = BatchNormalization()(dense1) # Helps stabilize training
    dense1 = Dropout(0.5)(dense1)         # Prevents overfitting

    dense2 = Dense(128, activation='relu')(dense1)
    dense2 = BatchNormalization()(dense2)
    dense2 = Dropout(0.5)(dense2)

    # Output layer
    # Has NUM_LABELS (28) neurons, one for each emotion
    # Uses 'sigmoid' activation, which is CRITICAL for multi-label.
    # It gives a 0-1 probability for *each* emotion independently.
    outputs = Dense(num_labels, activation='sigmoid')(dense2)

    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model
    # Uses 'binary_crossentropy' loss, which is CRITICAL for multi-label.
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=1e-3),
                  metrics=[
                      'AUC',
                      tf.keras.metrics.Precision(thresholds=0.5, name='precision'),
                      tf.keras.metrics.Recall(thresholds=0.5, name='recall')
                  ])

    return model


# ### 6.2. Build and Train the Model
# %%
print("Building Base CNN model...")
base_model = build_base_cnn_model()
base_model.summary()

# Callbacks for smart training
# Stop training if the validation loss doesn't improve for 3 epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# Reduce the learning rate if training plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

print("\n--- Starting Base Model Training ---")
print("This will take a few minutes per epoch...")

# Train the model
# NOTE: For a high-accuracy model, increase epochs to 20 or more.
# For this notebook, 5-10 epochs is a good balance of speed and performance.
history = base_model.fit(
    X_train_pad, y_train_base,
    epochs=50,  # <-- You can increase this for better performance
    batch_size=128,
    validation_data=(X_test_pad, y_test_base), # Use test set as validation
    callbacks=[early_stopping, reduce_lr]
)

print("\n--- Base Model Training Complete ---")


Building Base CNN model...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 100, 128)  │  2,560,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 100, 128)  │     49,280 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 100, 128)  │     65,664 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 100, 128)  │     82,048 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d[0][0]      │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_1[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_2[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 384)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │     98,560 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256)       │      1,024 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 28)        │      3,612 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,893,596 (11.04 MB)

 Trainable params: 2,892,828 (11.04 MB)

 Non-trainable params: 768 (3.00 KB)


--- Starting Base Model Training ---
This will take a few minutes per epoch...
Epoch 1/50
382/382 ━━━━━━━━━━━━━━━━━━━━ 29s 37ms/step - AUC: 0.5541 - loss: 0.6215 - precision: 0.0493 - recall: 0.3354 - val_AUC: 0.7932 - val_loss: 0.1456 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/50
382/382 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - AUC: 0.7741 - loss: 0.1482 - precision: 0.5795 - recall: 0.1005 - val_AUC: 0.8565 - val_loss: 0.1228 - val_precision: 0.7146 - val_recall: 0.2120 - learning_rate: 0.0010
Epoch 3/50
382/382 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - AUC: 0.8494 - loss: 0.1262 - precision: 0.6909 - recall: 0.2662 - val_AUC: 0.8860 - val_loss: 0.1136 - val_precision: 0.6480 - val_recall: 0.3542 - learning_rate: 0.0010
Epoch 4/50
382/382 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - AUC: 0.8946 - loss: 0.1097 - precision: 0.7607 - recall: 0.3493 - val_AUC: 0.8973 - val_loss: 0.1100 - val_precision: 0.6870 - val_recall: 0.3280 - learning_rate: 0.0010
Epoch 5/50


In [7]:
# ---
# ## Cell 7: Evaluate and Save the Base Model
# ---
# %%
print("Evaluating final model on the test set:")
loss, auc, precision, recall = base_model.evaluate(X_test_pad, y_test_base)

print(f"\nTest Loss: {loss:.4f}")
print(f"Test AUC: {auc:.4f}")
print(f"Test Precision (at 0.5 threshold): {precision:.4f}")
print(f"Test Recall (at 0.5 threshold): {recall:.4f}")


# ### 7.1. View Detailed Classification Report
# %%
# Get model predictions (as probabilities)
y_pred_probs = base_model.predict(X_test_pad)

# Convert probabilities to binary labels (0 or 1) using a 0.5 threshold
y_pred_binary = (y_pred_probs > 0.5).astype(int)

# Print a full classification report
# This shows precision, recall, and f1-score for *each* emotion
print("\n--- Detailed Classification Report (Test Set) ---")
print(classification_report(y_test_base, y_pred_binary, target_names=EMOTION_LABELS, zero_division=0))


# ### 7.2. Save Final Model Artifacts
# %%
# Save the trained base model
base_model.save('base_cnn_model.h5')
print("\n--- !!! PHASE 1 COMPLETE !!! ---")
print("Base model saved to 'base_cnn_model.h5'")
print("Tokenizer saved to 'tokenizer.json'")
print("\nThese two files are now ready for Phase 2.")


Evaluating final model on the test set:
170/170 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - AUC: 0.8989 - loss: 0.1091 - precision: 0.6919 - recall: 0.3340

Test Loss: 0.1100
Test AUC: 0.8973
Test Precision (at 0.5 threshold): 0.6870
Test Recall (at 0.5 threshold): 0.3280
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step



--- Detailed Classification Report (Test Set) ---
                precision    recall  f1-score   support

    admiration       0.67      0.52      0.58       504
     amusement       0.79      0.76      0.78       264
         anger       0.61      0.23      0.33       198
     annoyance       0.00      0.00      0.00       320
      approval       0.00      0.00      0.00       351
        caring       0.00      0.00      0.00       135
     confusion       0.00      0.00      0.00       153
     curiosity       0.00      0.00      0.00       284
        desire       0.00      0.00      0.00        83
disappointment       0.00      0.00      0.00       151
   disapproval       0.00      0.00      0.00       267
       disgust       0.95      0.15      0.25       123
 embarrassment       0.00      0.00      0.00        37
    excitement       0.00      0.00      0.00       103
          fear       0.00      0.00      0.00        78
     gratitude       0.95      0.89      0.92       

# Phase 2

In [8]:
# # Phase 2: Pseudo-Labeling via Reddit Scraping
#
# **Goal:** Use our "Base Model" to scrape and auto-label thousands of new, unlabeled Reddit comments.
# ---

# ### Cell 8: Install & Import New Libraries
#
# We need `praw` (the Reddit API library) and to re-import Keras/TensorFlow functions for loading our saved model.
# %%
import praw
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tqdm import tqdm  # A nice progress bar
import time

print("PRAW (Reddit API Wrapper) installed.")

PRAW (Reddit API Wrapper) installed.


In [9]:
# # Cell 10: (Phase 2) Configure and Load API Keys (V2 - Massive Scrape)
#
# This cell is updated to scrape over 100 subreddits to
# gather 80,000+ comments and find our 10,000+ pseudo-labels.
# %%
from google.colab import userdata

try:
    # --- Load keys securely from Colab Secrets ---
    REDDIT_CLIENT_ID = userdata.get('REDDIT_CLIENT_ID')
    REDDIT_CLIENT_SECRET = userdata.get('REDDIT_CLIENT_SECRET')
    # ---

    if REDDIT_CLIENT_ID is None or REDDIT_CLIENT_SECRET is None:
        raise ValueError("Could not find secrets. Did you add them correctly in the 'Key' menu?")

    print("Successfully loaded Reddit API keys from Secrets Manager.")

except Exception as e:
    print(f"--- !!! KEY ERROR !!! ---")
    print(f"Error loading secrets: {e}")
    print("Please click the 'Key' icon on the left sidebar and ensure you have added:")
    print("1. REDDIT_CLIENT_ID")
    print("2. REDDIT_CLIENT_SECRET")
    print("And make sure 'Notebook access' is toggled ON for both.")
    raise

# --- Standard Configuration ---
REDDIT_USER_AGENT = "EmotionScraper v1.0 by /u/your_reddit_username" # Change to your username

# --- 1. THE MASSIVE SUBREDDIT LIST ---
# We are targeting ~100 subreddits to get 80k+ comments
SUBREDDITS_TO_SCRAPE = [
    # India (General)
    'india', 'indiasocial', 'mumbai', 'delhi', 'bangalore', 'chennai', 'kolkata', 'Hyderabad',
    'IndianGaming', 'IndiaSpeaks',

    # India (Specific)
    'cricket', 'BollyBlindsNGossip', 'IndianCinema', 'IndianFood', 'bollywood',

    # General Positive (Joy, Gratitude, Love, Pride, Excitement)
    'MadeMeSmile', 'CongratsLikeImFive', 'UpliftingNews', 'wholesomememes', 'happy', 'dating',
    'love', 'marriage', 'todayiamhappy', 'aww', 'HumansBeingBros', 'toastme', 'GetMotivated',

    # General Negative (Anger, Annoyance, Disgust)
    'mildlyinfuriating', 'rant', 'unpopularopinion', 'offmychest', 'TrueOffMyChest', 'AmItheAsshole',
    'publicfreakout', 'facepalm', 'trashy', 'IdiotsInCars', 'ChoosingBeggars', 'rage',

    # General Negative (Sadness, Grief, Remorse, Fear)
    'depression', 'anxiety', 'confession', 'sad', 'lonely', 'mentalhealth', 'horror',
    'nosleep', 'creepy', 'whatsbotheringyou', 'griefsupport', 'ptsd', 'trauma',

    # Ambiguous (Curiosity, Confusion, Realization)
    'AskReddit', 'explainlikeimfive', 'NoStupidQuestions', 'Showerthoughts', 'mildlyinteresting',
    'whatisthisthing', 'Glitch_in_the_Matrix', 'self', 'findareddit', 'tipofmytongue', 'Damnthatsinteresting',

    # Global Topics (for neutral/mixed data)
    'worldnews', 'technology', 'gaming', 'movies', 'politics', 'science', 'Futurology',
    'personalfinance', 'tifu', 'books', 'LifeProTips', 'todayilearned', 'space', 'history',
    'philosophy', 'Cooking', 'sports', 'Music', 'videos', 'Art', 'gadgets', 'programming',
    'travel', 'news', 'environment', 'relationships', 'relationship_advice'
]

# --- 2. EFFICIENT LIMIT ---
# As you noted, the API only returns ~800-900.
# Setting the limit to 1000 is more efficient than 2000.
COMMENTS_PER_SUBREDDIT = 1500

# --- 3. CONFIDENCE THRESHOLD (YOUR CHOICE) ---
# To get 10,000+ labels, you will need to lower this.
# 0.95 = ~2.8% conversion (80k comments -> ~2.2k labels)
# 0.80 = ~12.5% conversion (80k comments -> ~10k labels)
#
# Lowering this gives you MORE data, but the labels will be
# less accurate. I recommend 0.85 or 0.80.
CONFIDENCE_THRESHOLD = 0.85

print(f"Configuration set. Will scrape {COMMENTS_PER_SUBREDDIT} comments from {len(SUBREDDITS_TO_SCRAPE)} subreddits.")
print(f"Targeting ~{len(SUBREDDITS_TO_SCRAPE) * 850} raw comments.")
print(f"Confidence threshold set to {CONFIDENCE_THRESHOLD * 100}%.")

Successfully loaded Reddit API keys from Secrets Manager.
Configuration set. Will scrape 1500 comments from 91 subreddits.
Targeting ~77350 raw comments.
Confidence threshold set to 85.0%.


In [10]:

# # Cell 11: (Phase 2) Load Phase 1 Artifacts
#
# This cell loads the `base_cnn_model.h5` and `tokenizer.json` files we just saved.
# %%
print("Loading Phase 1 artifacts...")

# Load the "brain"
try:
    base_model = load_model('base_cnn_model.h5')
    print("Successfully loaded 'base_cnn_model.h5'")
except Exception as e:
    print(f"--- !!! CRITICAL ERROR !!! ---")
    print(f"Could not load 'base_cnn_model.h5'. Error: {e}")
    print("Did you successfully run Cell 9 to save the model?")
    raise

# Load the "vocabulary"
try:
    with open('tokenizer.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
        tokenizer = tokenizer_from_json(data)
    print("Successfully loaded 'tokenizer.json'")
except Exception as e:
    print(f"--- !!! CRITICAL ERROR !!! ---")
    print(f"Could not load 'tokenizer.json'. Error: {e}")
    print("Did you successfully run Cell 6 to save the tokenizer?")
    raise

print("\n--- Model and Tokenizer are loaded and ready. ---")
base_model.summary()

Loading Phase 1 artifacts...
Successfully loaded 'base_cnn_model.h5'
Successfully loaded 'tokenizer.json'

--- Model and Tokenizer are loaded and ready. ---


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 100, 128)  │  2,560,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 100, 128)  │     49,280 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 100, 128)  │     65,664 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 100, 128)  │     82,048 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d[0][0]      │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_1[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_2[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 384)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │     98,560 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256)       │      1,024 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 28)        │      3,612 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,893,598 (11.04 MB)

 Trainable params: 2,892,828 (11.04 MB)

 Non-trainable params: 768 (3.00 KB)

 Optimizer params: 2 (12.00 B)

In [ ]:
from tqdm import tqdm

def scrape_reddit_data(keyword, max_posts=800, max_subreddits=10, posts_per_sub=100):
    """
    Scrapes Reddit for a given keyword (hashtag), automatically finding relevant subreddits.
    
    Parameters:
    - keyword: the search term / hashtag (string)
    - max_posts: approximate total posts to collect (int)
    - max_subreddits: maximum number of subreddits to search (int)
    - posts_per_sub: maximum posts per subreddit (int)
    
    Returns:
    - scraped_posts: list of dicts with keys: text, url, score, subreddit
    """
    if not reddit_client:
        print("Reddit client is not connected. Cannot scrape.")
        return []
    
    print(f"\nSearching for relevant subreddits for: '{keyword}' ...")
    
    # Step 1: Get relevant subreddits
    try:
        subreddits = [sub.display_name for sub in reddit_client.subreddits.search(keyword, limit=max_subreddits)]
        if not subreddits:
            print("No relevant subreddits found. Defaulting to 'all'.")
            subreddits = ["all"]
        else:
            print(f"Found {len(subreddits)} relevant subreddits: {subreddits}")
    except Exception as e:
        print(f"Error finding subreddits: {e}. Defaulting to 'all'.")
        subreddits = ["all"]

    # Step 2: Scrape posts from each subreddit
    scraped_posts = []
    for sub in subreddits:
        print(f"\nScraping up to {posts_per_sub} posts from r/{sub}...")
        try:
            for submission in tqdm(
                reddit_client.subreddit(sub).search(keyword, sort="relevant", limit=posts_per_sub),
                total=posts_per_sub
            ):
                post_text = submission.title + " " + submission.selftext
                if post_text.strip():  # skip empty posts
                    scraped_posts.append({
                        "text": post_text,
                        "url": submission.url,
                        "score": submission.score,
                        "subreddit": sub
                    })
                if len(scraped_posts) >= max_posts:
                    break
        except Exception as e:
            print(f"Error scraping r/{sub}: {e}. Skipping.")
        
        if len(scraped_posts) >= max_posts:
            print(f"\nReached max posts limit: {max_posts}")
            break
    
    print(f"\n--- Scraping Complete ---")
    print(f"Total posts collected: {len(scraped_posts)}")
    return scraped_posts

In [11]:

# # Cell 12: (Phase 2) Connect to Reddit & Scrape Data
# %%
def scrape_reddit_comments(client_id, client_secret, user_agent, sub_list, limit_per_sub):
    if "YOUR_CLIENT_ID_HERE" in client_id:
        print("--- !!! API KEY ERROR !!! ---")
        print("Please update your Reddit API keys in Cell 10 before running.")
        return []

    print("Connecting to Reddit API...")
    try:
        reddit = praw.Reddit(
            client_id=client_id,
            client_secret=client_secret,
            user_agent=user_agent
        )
        print("Successfully connected to Reddit.")
    except Exception as e:
        print(f"Failed to connect to Reddit: {e}")
        return []

    raw_comments = []

    for sub_name in sub_list:
        try:
            print(f"\n--- Scraping subreddit: r/{sub_name} ---")
            subreddit = reddit.subreddit(sub_name)

            for comment in tqdm(subreddit.comments(limit=limit_per_sub), total=limit_per_sub):
                if comment.body and comment.body != "[deleted]" and comment.body != "[removed]":
                    raw_comments.append(comment.body)

            print(f"Finished r/{sub_name}. Total comments scraped: {len(raw_comments)}")
            time.sleep(2) # Be polite to Reddit's API

        except Exception as e:
            print(f"Error scraping r/{sub_name}: {e}. Skipping.")

    return raw_comments

# --- Run the Scraper ---
raw_comments_list = scrape_reddit_comments(
    REDDIT_CLIENT_ID,
    REDDIT_CLIENT_SECRET,
    REDDIT_USER_AGENT,
    SUBREDDITS_TO_SCRAPE,
    COMMENTS_PER_SUBREDDIT
)

print(f"\n--- Scraping Complete ---")
print(f"Total raw comments collected: {len(raw_comments_list)}")

Connecting to Reddit API...
Successfully connected to Reddit.

--- Scraping subreddit: r/india ---


  0%|          | 0/1500 [00:00<?, ?it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

  0%|          | 1/1500 [00:01<25:29,  1.02s/it]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

  7%|▋         | 101/1500 [00:01<00:18, 73.67it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-a

Finished r/india. Total comments scraped: 882

--- Scraping subreddit: r/indiasocial ---


  0%|          | 0/1500 [00:00<?, ?it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

  0%|          | 1/1500 [00:00<17:11,  1.45it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

  7%|▋         | 101/1500 [00:01<00:14, 94.10it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-a

Finished r/indiasocial. Total comments scraped: 1861

--- Scraping subreddit: r/mumbai ---


...

Finished r/environment. Total comments scraped: 82376

--- Scraping subreddit: r/relationships ---


  0%|          | 0/1500 [00:00<?, ?it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

  0%|          | 1/1500 [00:00<18:43,  1.33it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

  7%|▋         | 101/1500 [00:01<00:16, 83.80it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-a

Finished r/relationships. Total comments scraped: 83261

--- Scraping subreddit: r/relationship_advice ---


  0%|          | 0/1500 [00:00<?, ?it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

  0%|          | 1/1500 [00:00<17:29,  1.43it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

  7%|▋         | 101/1500 [00:01<00:18, 74.62it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-a

Finished r/relationship_advice. Total comments scraped: 84195

--- Scraping Complete ---
Total raw comments collected: 84195


Processing new raw comments...
Step 1/4: Cleaning text...


100%|██████████| 84195/84195 [00:19<00:00, 4353.41it/s]


Step 2/4: Tokenizing text...
Step 3/4: Padding sequences...
Step 4/4: Running 'Emotion Expert' model on new data...
329/329 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step

--- Prediction Complete ---
Analyzed 84195 comments.
Probabilities array shape: (84195, 28)


In [13]:

# # Cell 14: (Phase 2) Filter for High-Confidence Pseudo-Labels
# %%
print(f"Filtering {all_probabilities.shape[0]} predictions with a {CONFIDENCE_THRESHOLD*100}% threshold...")

new_pseudo_data = []

# Loop through every comment's probability array
for i in range(len(all_probabilities)):
    # Get the 28 probabilities for this one comment
    probs = all_probabilities[i]

    # Find all emotion indices that are *above* our threshold
    # (e.g., [1, 17, 27] which corresponds to amusement, joy, neutral)
    confident_indices = np.where(probs > CONFIDENCE_THRESHOLD)[0]

    # We only save this sample if the model was confident about *at least one* emotion
    if len(confident_indices) > 0:

        # We need to save the *original* cleaned text
        text_to_save = cleaned_new_comments[i]

        # We need to save the labels in the *same format* as our original data
        # e.g., "1,17,27"
        label_str_to_save = ",".join([str(idx) for idx in confident_indices])

        new_pseudo_data.append({
            "cleaned_text": text_to_save,
            "labels": label_str_to_save
        })

print(f"\n--- Filtering Complete ---")
print(f"Original unlabeled comments: {len(all_probabilities)}")
print(f"New high-confidence pseudo-labels found: {len(new_pseudo_data)}")

df_pseudo = pd.DataFrame(new_pseudo_data)

print("\n--- Sample of new Pseudo-Labeled Data ---")
print(df_pseudo.head())

Filtering 84195 predictions with a 85.0% threshold...

--- Filtering Complete ---
Original unlabeled comments: 84195
New high-confidence pseudo-labels found: 9379

--- Sample of new Pseudo-Labeled Data ---
                                        cleaned_text labels
0  bruh even speak hindi lmao recent famous meme lol      1
1  incredibly insightful thank much sharing actua...     15
2  cool sound definitely check volunteer meet ama...      0
3  travelled bit chose mulki cause volunteered su...     27
4                                       calm pinkesh     27


In [14]:
# # Cell 15: (Phase 2) Combine Original & Pseudo-Data
#
# This cell combines our original human-labeled data (from Cell 5)
# with our new pseudo-labeled data (from Cell 14)
# and saves it to a single, final file.
# %%
print("Combining datasets...")

# --- FIX: Use the correct variable and column names ---

# 1. Get the original data (which is still in memory from Cell 5)
# The variable is 'df_base_train' and the columns are 'cleaned_text' and 'labels'
df_original_final = df_base_train[['cleaned_text', 'labels']]

# 2. Get the pseudo-labeled data (from Cell 14)
# This variable is already correct
df_pseudo_final = df_pseudo
# --- END FIX ---

# 3. Concatenate the two DataFrames
df_final_combined = pd.concat([df_original_final, df_pseudo_final], ignore_index=True)

# 4. Save the final "super-dataset"
output_filename = 'final_combined_dataset.csv'
df_final_combined.to_csv(output_filename, index=False, encoding='utf-8')

print("\n--- !!! PHASE 2 COMPLETE !!! ---")
print(f"Original human-labeled rows: {len(df_original_final)}")
print(f"New pseudo-labeled rows: {len(df_pseudo_final)}")
print(f"Total rows in new dataset: {len(df_final_combined)}")
print(f"\nFinal combined dataset saved to: '{output_filename}'")

Combining datasets...

--- !!! PHASE 2 COMPLETE !!! ---
Original human-labeled rows: 48836
New pseudo-labeled rows: 9379
Total rows in new dataset: 58215

Final combined dataset saved to: 'final_combined_dataset.csv'


# Phase 3 & 4

In [15]:
from mealpy.utils.space import IntegerVar, FloatVar

In [16]:

# # Cell 16: (Phase 3 & 4) Load Combined Dataset & Final Prep
# %%

# --- 1. Load Tokenizer & MLB (from Phase 1) ---
with open('tokenizer.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

    # --- FIX: Get VOCAB_SIZE directly from the tokenizer ---
    # The 'config' attribute does not exist.
    VOCAB_SIZE = tokenizer.num_words
    # MAX_LEN was defined as a global variable in Cell 4
    MAX_LEN = 100
    # --- END FIX ---

mlb = MultiLabelBinarizer(classes=range(NUM_LABELS))
mlb.fit(None)

# --- 2. Load the "Super-Dataset" ---
try:
    df_final = pd.read_csv('final_combined_dataset.csv')
    df_final.dropna(inplace=True) # Drop any rows with missing data
except FileNotFoundError:
    print("--- !!! CRITICAL ERROR !!! ---")
    print("Could not find 'final_combined_dataset.csv'. Did Cell 15 run successfully?")
    raise

# --- 3. Process Text and Labels ---
def process_final_labels(label_str):
    if not isinstance(label_str, str) or label_str.strip() == '':
        label_indices = []
    else:
        label_indices = [int(i) for i in label_str.split(',') if i.isdigit()]
    return mlb.transform([label_indices])[0]

print("Processing final dataset...")
X_final = df_final['cleaned_text']
y_final_labels = df_final['labels']
y_final_vectors = np.array(y_final_labels.apply(process_final_labels).tolist())

print("Vectorizing and padding final text...")
X_final_pad = pad_sequences(tokenizer.texts_to_sequences(X_final), maxlen=MAX_LEN, padding='post', truncating='post')

# --- 4. Final Train-Validation-Test Split ---
print("Creating final Train, Validation, and Test splits...")
# We'll do a 70% Train, 15% Validation, 15% Test split.
X_train, X_temp, y_train, y_temp = train_test_split(
    X_final_pad, y_final_vectors, test_size=0.3, random_state=42
)

# Split the 30% 'temp' set in half to get 15% val and 15% test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

print(f"Total samples: {len(X_final_pad)}")
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Test samples: {len(X_test)}")

Processing final dataset...
Vectorizing and padding final text...
Creating final Train, Validation, and Test splits...
Total samples: 57907
Training samples: 40534
Validation samples: 8686
Test samples: 8687


In [17]:
# ### Cell 17: Define the AOA Fitness Function (V3 - CNN Only)
#
# This is our new, correct fitness function.
# It REMOVES the KNN and allows the AOA to optimize
# the CNN's core hyperparameters.
# %%

def build_AOA_cnn_model(num_filters, kernel_size, learning_rate):
    """
    Builds a *complete* Keras CNN model for classification.
    """
    inputs = Input(shape=(MAX_LEN,))
    embedding = Embedding(input_dim=VOCAB_SIZE, output_dim=128, input_length=MAX_LEN)(inputs)

    # Use the hyperparameters from the AOA
    conv1 = Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu', padding='same')(embedding)
    pool1 = GlobalMaxPooling1D()(conv1)

    conv2 = Conv1D(filters=num_filters, kernel_size=kernel_size+1, activation='relu', padding='same')(embedding)
    pool2 = GlobalMaxPooling1D()(conv2)

    conv3 = Conv1D(filters=num_filters, kernel_size=kernel_size+2, activation='relu', padding='same')(embedding)
    pool3 = GlobalMaxPooling1D()(conv3)

    merged = tf.keras.layers.concatenate([pool1, pool2, pool3], axis=1)

    dense1 = Dense(256, activation='relu')(merged)
    dense1 = BatchNormalization()(dense1)
    dense1 = Dropout(0.5)(dense1)

    dense2 = Dense(128, activation='relu')(dense1)
    dense2 = BatchNormalization()(dense2)
    dense2 = Dropout(0.5)(dense2)

    # This is the final classifier layer
    outputs = Dense(NUM_LABELS, activation='sigmoid')(dense2)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=learning_rate), # <-- Use the AOA learning rate
        metrics=[tf.keras.metrics.AUC(name='auc', multi_label=True)]
    )

    return model


def fitness_function(solution):
    """
    Trains and evaluates the full CNN-ONLY pipeline.
    """
    # 1. Unpack Hyperparameters
    num_filters = int(solution[0])
    kernel_size = int(solution[1])
    learning_rate = float(solution[2])

    if num_filters < 16: num_filters = 16
    if kernel_size < 2: kernel_size = 2

    print(f"\n--- Testing Solution: Filters={num_filters}, Kernel={kernel_size}, LR={learning_rate:.6f} ---")

    try:
        # 2. Build the CNN
        model = build_AOA_cnn_model(num_filters, kernel_size, learning_rate)

        # 3. Calculate Class Weights for this training fold
        total_counts = np.sum(y_train, axis=0)
        total_samples = len(y_train)
        num_classes = len(EMOTION_LABELS)

        fold_class_weights = {}
        for i in range(num_classes):
            if total_counts[i] > 0:
                fold_class_weights[i] = (total_samples / (num_classes * total_counts[i]))
            else:
                fold_class_weights[i] = 1.0

        # 4. Train the CNN (on GPU)
        model.fit(
            X_train, y_train,
            epochs=3,  # Low epoch count for fast optimization
            batch_size=256,
            validation_data=(X_val, y_val),
            callbacks=[EarlyStopping(monitor='val_loss', patience=1)],
            class_weight=fold_class_weights, # <-- PASS THE WEIGHTS
            verbose=0  # 0 = silent
        )

        # 5. Evaluate the CNN directly
        print("Evaluating CNN on validation set...")
        y_pred_probs = model.predict(X_val, verbose=0, batch_size=256)

        # We'll use a 0.3 threshold for our score (to match our final report)
        y_pred_binary = (y_pred_probs > 0.3).astype(int)

        # We use 'micro' F1-score as it's a good all-around metric
        score = f1_score(y_val, y_pred_binary, average='micro')

        print(f"--- Solution Score (Micro-F1 @ 0.3): {score:.4f} ---")

        # AOA tries to *minimize* the score, so we return 1.0 - our F1-score
        return 1.0 - score

    except Exception as e:
        print(f"--- !!! Error in solution: {e} !!! ---")
        return 1.0

print("Fitness function defined (AOA-Optimized CNN-Only).")

Fitness function defined (AOA-Optimized CNN-Only).


In [18]:
# ### Cell 18: Configure & Run AOA (V3 - CNN Only)
# %%
print("Configuring AOA Optimizer...")

# Define the "problem" using the correct 'obj_func' key and 'bounds'
problem_dict = {
    "obj_func": fitness_function,
    "bounds": [
        IntegerVar(lb=16, ub=128, name="num_filters"),
        IntegerVar(lb=2, ub=7, name="kernel_size"),
        FloatVar(lb=1e-4, ub=1e-2, name="learning_rate") # <-- Optimize learning rate
    ],
    "minmax": "min",
    "log_to": None,
    "save_population": False,
}

# Set optimizer parameters
AOA_EPOCHS = 3
AOA_POP_SIZE = 10

model = OriginalAOA(epoch=AOA_EPOCHS, pop_size=AOA_POP_SIZE)

print(f"Starting AOA optimization... ({AOA_EPOCHS} epochs, {AOA_POP_SIZE} population size)")
print("This will be very slow. Please be patient.")

best_agent = model.solve(problem_dict)

best_solution = best_agent.solution
best_fitness = best_agent.target.fitness

print("\n--- AOA Optimization Complete ---")
print(f"Best fitness (error) score: {best_fitness:.4f}")

# Unpack the best solution
best_filters = int(best_solution[0])
best_kernel = int(best_solution[1])
best_lr = float(best_solution[2])

print(f"Best Hyperparameters Found:")
print(f"  - CNN Filters: {best_filters}")
print(f"  - CNN Kernel Size: {best_kernel}")
print(f"  - CNN Learning Rate: {best_lr:.6f}")

Configuring AOA Optimizer...
Starting AOA optimization... (3 epochs, 10 population size)
This will be very slow. Please be patient.

--- Testing Solution: Filters=103, Kernel=3, LR=0.008858 ---
Evaluating CNN on validation set...
--- Solution Score (Micro-F1 @ 0.3): 0.2614 ---

--- Testing Solution: Filters=21, Kernel=4, LR=0.006828 ---
Evaluating CNN on validation set...
--- Solution Score (Micro-F1 @ 0.3): 0.3521 ---

--- Testing Solution: Filters=83, Kernel=7, LR=0.005066 ---
Evaluating CNN on validation set...
--- Solution Score (Micro-F1 @ 0.3): 0.2786 ---

--- Testing Solution: Filters=73, Kernel=2, LR=0.002102 ---
Evaluating CNN on validation set...
--- Solution Score (Micro-F1 @ 0.3): 0.2494 ---

--- Testing Solution: Filters=24, Kernel=6, LR=0.008294 ---
Evaluating CNN on validation set...
--- Solution Score (Micro-F1 @ 0.3): 0.3320 ---

--- Testing Solution: Filters=121, Kernel=4, LR=0.006547 ---
Evaluating CNN on validation set...
--- Solution Score (Micro-F1 @ 0.3): 0.3163 

In [19]:
# ### Cell 19: Train Final Model with Best Parameters (V3 - CNN Only)
# %%
print("--- Training Final Model ---")
print(f"Using: Filters={best_filters}, Kernel={best_kernel}, LR={best_lr:.6f}")

# 1. Combine training and validation sets
X_train_full = np.concatenate((X_train, X_val), axis=0)
y_train_full = np.concatenate((y_train, y_val), axis=0)

print(f"Full training set size: {len(X_train_full)}")
print(f"Final test set size: {len(X_test)}")

# 2. Calculate class weights for the *full* training set
total_counts = np.sum(y_train_full, axis=0)
total_samples = len(y_train_full)
num_classes = len(EMOTION_LABELS)
final_class_weights = {}
for i in range(num_classes):
    if total_counts[i] > 0:
        final_class_weights[i] = (total_samples / (num_classes * total_counts[i]))
    else:
        final_class_weights[i] = 1.0

# 3. Build the final CNN *classifier* with the *best* params
# (The 'build_AOA_cnn_model' function is still in memory from Cell 17)
final_model = build_AOA_cnn_model(best_filters, best_kernel, best_lr)

# 4. Train the final CNN on the *full* dataset
print("\nTraining final CNN...")
final_model.fit(
    X_train_full, y_train_full,
    epochs=15,  # More epochs for the final model
    batch_size=256,
    callbacks=[EarlyStopping(monitor='loss', patience=3)],
    class_weight=final_class_weights, # Pass final weights
    verbose=1
)

# 5. Evaluate on the unseen test set
print("\n--- Final Model Evaluation (on Test Set) ---")
y_pred_probs = final_model.predict(X_test, verbose=0, batch_size=256)

# Use a 0.3 threshold for our final report (to match the fitness function)
y_pred_binary = (y_pred_probs > 0.3).astype(int)

print("\n--- Detailed Classification Report (Test Set) ---")
print(classification_report(y_test, y_pred_binary, target_names=EMOTION_LABELS, zero_division=0))

--- Training Final Model ---
Using: Filters=128, Kernel=1, LR=0.005371
Full training set size: 49220
Final test set size: 8687

Training final CNN...
Epoch 1/15
193/193 ━━━━━━━━━━━━━━━━━━━━ 19s 39ms/step - auc: 0.5242 - loss: 0.3510
Epoch 2/15
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - auc: 0.7069 - loss: 0.1245
Epoch 3/15
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - auc: 0.8106 - loss: 0.1070
Epoch 4/15
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - auc: 0.8560 - loss: 0.0945
Epoch 5/15
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - auc: 0.8815 - loss: 0.0863
Epoch 6/15
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - auc: 0.8998 - loss: 0.0777
Epoch 7/15
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - auc: 0.9107 - loss: 0.0712
Epoch 8/15
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - auc: 0.9246 - loss: 0.0649
Epoch 9/15
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - auc: 0.9305 - loss: 0.0616
Epoch 10/15
193/193 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - auc: 0.9398 - loss: 0.0585
Epoch 11/15
193/193 ━━━━━━━━

In [21]:
# # Cell 20: (Phase 3 & 4) Save Final Model Artifacts (V3 - CNN Only)
#
# This cell saves your final model in *both* the new .keras format
# and the legacy .h5 format, as requested.
# %%

print("Saving final model artifacts...")

# 1. Save the Final CNN Classifier in the modern .keras format (Recommended)
# 'final_model' is the variable from Cell 19
final_model.save('final_cnn_model.keras')
print("Final CNN classifier saved to 'final_cnn_model.keras' (recommended format)")

# 2. Save the Final CNN Classifier in the legacy .h5 format (As requested)
try:
    final_model.save('final_cnn_model.h5')
    print("Final CNN classifier also saved to 'final_cnn_model.h5'")
except Exception as e:
    print(f"Error saving to .h5 format: {e}")
    print("This can sometimes happen. The '.keras' file is the primary backup.")

# 3. Save the Tokenizer
tokenizer_json = tokenizer.to_json()
with open('final_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))
print("Final Tokenizer saved to 'final_tokenizer.json'")

print("\n--- !!! ALL PHASES COMPLETE !!! ---")
print("You are now ready to build your app with 'final_tokenizer.json' and a model file.")

Saving final model artifacts...
Final CNN classifier saved to 'final_cnn_model.keras' (recommended format)
Final CNN classifier also saved to 'final_cnn_model.h5'
Final Tokenizer saved to 'final_tokenizer.json'

--- !!! ALL PHASES COMPLETE !!! ---
You are now ready to build your app with 'final_tokenizer.json' and a model file.
